Semantic Analysis & Finetuning Bert
===
MAIC - Winter, Week 3<br>
{ holiday workshop }
```
  ___________
 /0   /  \   \
/  \ M A I C /\
\ / *    /  / /
 \___\__/__@ /
           \/
```

Santa Claus recently heard about the recent advents of AI, and has come to MAIC asking for some help.

Problem: how do we tell if someone is naughty or nice?<br>
We could use sentiment analysis and finetuning...

#### Sentiment Analysis
How could you measure qualitative and subjective information in a piece of text?<br>
It may initially seem simple, but there are many exceptions and edge cases...

- Simple case : "MSOE is the best school."
- Double negatives : "I don't dislike this class."
- Sarcasm : "I love getting 5 hours of laborious class work."
- Difficult to catagorize : "I didn't like taking CS 1010, but I would recommend it for future students."

It can go past "good" and "bad"...

- Query : "How can I take independent study?"
- Suggestion : "You should take MA 100 instead of MA 300."
- Opinion : "CS 3040 is the best class."

It can go past categories...

- 10% intensity : "I don't dislike this class."
- 40% intensity : "I like this class."
- 80% intensity : "This is one of my favorite classes."
- 95% intensity : "This is litterally the best I've ever taken by far."

We will be categorizing things as _positive_ or _negative_ (_naughty_ or _nice_).<br>
To account for intensity, each category will have an extreme variant, and there will be a neutral category:<br>
- Extremely Naughty/Negative
- Naughty/Negative
- Neutral 
- Nice/Positive
- Extremely Nice/Positive

#### Finetuning & Bert
How will we actually go about doing this?

We will take an existing model (Bert), and _finetune_ it.<br>
What are Berts? They are models from a collection of versatile language models that use transformers. They were made by Google in 2018, and they are also free to use in your projects!<br>
You can learn more here (https://tfhub.dev/google/collections/bert/1), but we'll just be treating it as a "black-box" which takes text and gives an embeding with 255 values.

Finetuning is the process of taking an existing model (and its weights!), and training it _more_ on data that's specific to your problem. We will be finetuning bert on our yet-to-be-discussed data.

In [ ]:
# UNCOMMENT if there is a logging error with pip
# !pip config set global.disable-pip-version-check true

# UNCOMMENT if you don't have tensorflow_hub or tensorflow_text installed. (If you don't know, then uncomment)
# !pip install tensorflow_hub
# !pip install tensorflow-text

# UNCOMMENT if no permission to default directory used by tensorflow_hub
# import os ; os.environ['TFHUB_CACHE_DIR']='.cache/tfhub_modules'

# Might have to restart kernel after installing stuff #

In [ ]:
import pandas as pd
import numpy as np
import re

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow.keras import layers
from matplotlib import pyplot as plt

The data we're using: https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification

This is Twitter data from the beginning of the covid pandemic.<br>
It includes (among other things) the tweet-text, and a sentiment value (we will explore the data shortly).

In [ ]:
NUM_SAMPLES = 10000
df = pd.read_csv('/data/ai_club/util/datasets/holiday-sentiment-analysis/Corona_NLP_train.csv', encoding='ISO-8859-1',
                nrows=NUM_SAMPLES)

In [ ]:
df

In [ ]:
np.unique(df['Sentiment'])

In [ ]:
sentiments = ['Extremely Negative',  'Negative', 'Neutral', 'Positive', 'Extremely Positive']

def ohe_sentiment(sentiment):
    return np.eye(5)[
        sentiments.index(sentiment)
    ]

In [ ]:
ohe_sentiment('Extremely Negative')

In [ ]:
def clean_text(text):
    return re.sub('http(s?)://([^\s]*)', '', # no links
           re.sub('#([^\s]+)', '\\1',        # remove '#' from hashtag
           re.sub('@([^\s]+)', '', text)))   # remove '@user'

In [ ]:
clean_text('test #test test #hashtag https://253253 @user')

In [ ]:
train_x = np.array([clean_text(t) for t in df['OriginalTweet']])
train_y = np.array([ohe_sentiment(m) for m in df['Sentiment']])

In [ ]:
idx = 8005
print(
    train_x[idx],
    train_y[idx]
)

In [ ]:
df_valid = pd.read_csv('/data/ai_club/util/datasets/holiday-sentiment-analysis/Corona_NLP_test.csv', encoding='ISO-8859-1',
                nrows=NUM_SAMPLES)
valid_x = np.array([clean_text(t) for t in df['OriginalTweet']])
valid_y = np.array([ohe_sentiment(m) for m in df['Sentiment']])

In [ ]:
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/2",
    trainable=False)

# How does this work?
# (What is pooled output?) https://www.tensorflow.org/text/tutorials/classify_text_with_bert
# https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/2

---

**What exactly is bert at the python-level of things?**

In [ ]:
p = preprocessor(['test'])
b = bert(p)

What do `p`, `b`, `bert`, and `preprocessor` look like?

In [ ]:
# preprocessor
# bert
# p
# b

---

**Our actual model**

In [ ]:
input = layers.Input(shape=(), dtype=tf.string)
x = bert(preprocessor(input))['pooled_output']
x = layers.Dense(5, activation='softmax')(x)

model = tf.keras.Model(input, x)
model.summary()

In [ ]:
bert.trainable=True

model.compile(
    optimizer=tf.keras.optimizers.Adam(3e-5),
    loss="categorical_crossentropy", 
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_x,
    train_y,
    epochs=7,
    validation_data=(valid_x, valid_y)
)
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['accuracy'], label='accuracy')
plt.xlabel('epoch')
plt.legend()
plt.show()

---

**How to use the model...**

In [ ]:
pred = model.predict(['Example text 1', 'Example text 2'])

In [ ]:
pred

In [ ]:
def see_prediction(prediction):
    idx = np.argmax(prediction)
    value = prediction[idx]
    result = sentiments[idx].replace('Positive', 'Nice').replace('Negative', 'Naughty')
    print(f'- - - {result} - - -\nwith a probability of {value}')
    plt.bar(sentiments, prediction)
    plt.tight_layout()

In [ ]:
pred = model.predict([''])

In [ ]:
see_prediction(pred[0])

---

**Resources**

- About sentiment analysis (https://www.freecodecamp.org/news/what-is-sentiment-analysis-a-complete-guide-to-for-beginners/)
- Training Bert to do seniment analysis on IMBD data (https://www.tensorflow.org/text/tutorials/classify_text_with_bert)
- Different Bert models you can use (https://tfhub.dev/google/collections/bert/1)

---

**Challenge Problems**

<span style="color:green">&#9646;&#9646;</span><br>
(1 challenge point)<br>
Train a larger bert model on the same data and determine whether the results are better. (https://tfhub.dev/google/collections/bert/1)<br>
<br><br>
<span style="color:green">&#9646;&#9646;</span><br>
(1 challenge point)<br>
Train this model, and use it to classify new data into lists of _naughty_ and _nice_ strings of text. This "new data" can be a subset of the Twitter data that you don't train the model on.<br>
<br><br>
<span style="color:green">&#9646;&#9646;</span><span style="color:#eedd00">&#9646;&#9646;</span><br>
(2 challenge points)<br>
Finetune Bert to perform sentiment analysis on a new dataset, such as the IMDB dataset. (https://www.tensorflow.org/text/tutorials/classify_text_with_bert)<br>
<br><br>
<span style="color:green">&#9646;&#9646;</span><span style="color:#eedd00">&#9646;&#9646;</span><span style="color:red">&#9646;&#9646;</span><br>
(3 challenge points)<br>
Further clean the Twitter dataset by spell-checking words and correcting them. Here is a python library that can perform the task of spell-correction: (https://pypi.org/project/pyspellchecker/)<br>
<br><br>
<span style="color:green">&#9646;&#9646;</span><span style="color:#eedd00">&#9646;&#9646;</span><span style="color:red">&#9646;&#9646;</span><span style="color:magenta">&#9646;&#9646;</span><br>
(4 challenge points)<br>
Perform sentiment analysis with categories that are more complex. E.G., _query_ vs _response_.<br>
For instance, this Kaggle dataset includes job descriptions and whether they're real or fake (https://www.kaggle.com/datasets/shivamb/real-or-fake-fake-jobposting-prediction).<br>
<br><br><br><br>
You can ask the eboard any questions